#######

使用续费率的数据实现了wide & deep模型    
使用的api为contrib,可能面临被频繁替换的危险， 因此在tf_WD_core中重新改写api


结果：预测续费的的pr_auc达到0.938，但是没有解释性


#######

-----------------

In [3]:
import tensorflow as tf
import pandas as pd
import tempfile

In [22]:
# import data
df_all = pd.read_csv('data/all.csv').drop(['Unnamed: 0'], axis=1)
df_part = pd.concat([df_all.iloc[:, :23], df_all.iloc[:, 71:]], axis=1)

In [24]:
# categorical base column(5)——离散类别列
visa_typ_cd = tf.contrib.layers.sparse_column_with_keys(
    column_name="visa_typ_cd", 
    keys=["A", "A1", "A2", "A3", "B", "C", "D1", "D2", "D3"])
top_schl_id = tf.contrib.layers.sparse_column_with_keys(
    column_name="top_schl_id", 
    keys=["432466", "430552", "431016", "432643", "434410",
       "432715", "434200", "409", "432710", "430564",
       "432699", "436223", "434266", "437282", "432468",
       "439174", "435391", "432590", "432791", "437357",
       "438276", "436224", "435620", "430558", "432610",
       "439177", "437353", "438456", "430627", "439175",
       "437317", "437367", "438445", "435300", "436070",
       "437414", "435590", "439176", "435467", "440467"])
year = tf.contrib.layers.sparse_column_with_keys(
    column_name="year", 
    keys=["2013", "2014", "2015", "2016", "2017", "2018"])
city = tf.contrib.layers.sparse_column_with_keys(
    column_name="city", 
    keys=["changchun", "shanghai", "beijin", "zhuhai", "yantai", "changsha",
        "chongqing", "yangzhou", "linfen", "taizhou", "wenzhou", "wuhan",
        "jiamusi", "nantong", "wuxi", "hangzhou", "dalian", "huzhou",
        "shenzheng", "xian", "ningbo", "changzhou"])
level = tf.contrib.layers.sparse_column_with_keys(
    column_name="level", 
    keys=["2nd", "1st", "new1", "4th", "3rd"])



# continues column(27)——数值列
par_days = tf.contrib.layers.real_valued_column("par_days")
actual_day = tf.contrib.layers.real_valued_column("actual_day")
sub_tol = tf.contrib.layers.real_valued_column("sub_tol")
sub_avg = tf.contrib.layers.real_valued_column("sub_avg")
sub_avg_a = tf.contrib.layers.real_valued_column("sub_avg_a")
ex_tol = tf.contrib.layers.real_valued_column("ex_tol")
ex_subrate = tf.contrib.layers.real_valued_column("ex_subrate")
sub_ins_sd = tf.contrib.layers.real_valued_column("sub_ins_sd")
sub_ins_m = tf.contrib.layers.real_valued_column("sub_ins_m")
ins_m = tf.contrib.layers.real_valued_column("ins_m")
auc_sub_avg = tf.contrib.layers.real_valued_column("auc_sub_avg")
auc_sub_sd = tf.contrib.layers.real_valued_column("auc_sub_sd")
ins_max = tf.contrib.layers.real_valued_column("ins_max")
ins_summit_four_1 = tf.contrib.layers.real_valued_column("ins_summit_four_1")
ins_summit_four_2 = tf.contrib.layers.real_valued_column("ins_summit_four_2")
ins_summit_four_3 = tf.contrib.layers.real_valued_column("ins_summit_four_3")
ins_summit_max = tf.contrib.layers.real_valued_column("ins_summit_max")
notscWe_sub = tf.contrib.layers.real_valued_column("notscWe_sub")
notscWe_ex = tf.contrib.layers.real_valued_column("notscWe_ex")
notscWd_sub = tf.contrib.layers.real_valued_column("notscWd_sub")
notscWd_ex = tf.contrib.layers.real_valued_column("notscWd_ex")
sc_sub = tf.contrib.layers.real_valued_column("sc_sub")
sc_ex = tf.contrib.layers.real_valued_column("sc_ex")
pub_avg = tf.contrib.layers.real_valued_column("pub_avg")
pub_sd = tf.contrib.layers.real_valued_column("pub_sd")
n_ex = tf.contrib.layers.real_valued_column("n_ex")
n_pkg = tf.contrib.layers.real_valued_column("n_pkg")
n_pra_ex_avg = tf.contrib.layers.real_valued_column("n_pra_ex_avg")
n_pra_ex_sd = tf.contrib.layers.real_valued_column("n_pra_ex_sd")
delta_days = tf.contrib.layers.real_valued_column("delta_days")


# bucketized categorical column(3)——分桶列
par_days_buckets = tf.contrib.layers.bucketized_column(
    par_days, 
    boundaries=[7, 31, 93, 183, 365, 730])
actual_day_buckets = tf.contrib.layers.bucketized_column(
    actual_day, 
    boundaries=[5, 25, 75, 150, 300, 600])
delta_days_buckets = tf.contrib.layers.bucketized_column(
    delta_days, 
    boundaries=[7, 31, 93, 183, 365, 730, 1650])

# define wide features
wide_columns = [
    visa_typ_cd, 
    city,
    top_schl_id, year, level, par_days_buckets, 
    actual_day_buckets, delta_days_buckets,
    tf.contrib.layers.crossed_column([visa_typ_cd, top_schl_id], hash_bucket_size=int(1e4)),
    tf.contrib.layers.crossed_column([visa_typ_cd, year], hash_bucket_size=int(1e4)),
    tf.contrib.layers.crossed_column([visa_typ_cd, city], hash_bucket_size=int(1e4)),
    tf.contrib.layers.crossed_column([visa_typ_cd, level], hash_bucket_size=int(1e4)),
    tf.contrib.layers.crossed_column([visa_typ_cd, top_schl_id, year], hash_bucket_size=int(1e4)),
    tf.contrib.layers.crossed_column([year, top_schl_id], hash_bucket_size=int(1e4))    
]

# define deep features
# embedding的维度越高，这个模型不得不学习的特征表示的自由度越大。基于经验，一般选择log2(n) 【以2为底】,n是一个特征列中唯一特征的数量
deep_columns = [
   par_days, actual_day, sub_tol, sub_avg, sub_avg_a, ex_tol, ex_subrate, sub_ins_sd, 
   sub_ins_m, ins_m, auc_sub_avg, auc_sub_sd, ins_max, ins_summit_four_1, ins_summit_four_2,
   ins_summit_four_3, ins_summit_max, notscWe_sub, notscWe_ex, notscWd_sub, notscWd_ex, 
   sc_sub, sc_ex, pub_avg, pub_sd, n_ex, n_pkg, n_pra_ex_avg, n_pra_ex_sd, delta_days,
   tf.contrib.layers.embedding_column(visa_typ_cd, dimension=3),
   tf.contrib.layers.embedding_column(top_schl_id, dimension=5),
   tf.contrib.layers.embedding_column(year, dimension=3),
   tf.contrib.layers.embedding_column(city, dimension=5),
   tf.contrib.layers.embedding_column(level, dimension=3)
]

# 创建临时文件存储过程数据
model_dir = tempfile.mkdtemp() # create a temp path
m = tf.contrib.learn.DNNLinearCombinedClassifier(
    model_dir=model_dir, 
    linear_feature_columns=wide_columns,
    dnn_feature_columns=deep_columns,
    dnn_hidden_units=[100,60,30]   
)


# define the column names for datasets
COLUMNS = [
        'visa_typ_cd', 
        'city'
        'par_days', 'actual_day', 'sub_tol',
        'sub_avg', 'sub_avg_a', 'ex_tol', 'ex_subrate', 'sub_ins_sd',
        'sub_ins_m', 'ins_m', 'auc_sub_avg', 'auc_sub_sd', 'ins_max',
        'ins_summit_four_1', 'ins_summit_four_2', 'ins_summit_four_3',
        'ins_summit_max', 'notscWe_sub', 'notscWe_ex', 'notscWd_sub',
        'notscWd_ex', 'sc_sub', 'sc_ex', 'pub_avg', 'pub_sd', 'n_ex',
        'n_pkg', 'n_pra_ex_avg', 'n_pra_ex_sd', 'top_schl_id',
        'year', 'level','delta_days']

LABEL_COLUMN = 'y'

CATEGORY_COLUMNS = [
    'visa_typ_cd',
    'city',
    'top_schl_id','year', 'level']

CONTINUES_COLUMNS = ['par_days', 'actual_day', 'sub_tol',
       'sub_avg','sub_avg_a', 'ex_tol', 'ex_subrate', 'sub_ins_sd',
       'sub_ins_m', 'ins_m', 'auc_sub_avg', 'auc_sub_sd', 'ins_max',
       'ins_summit_four_1', 'ins_summit_four_2', 'ins_summit_four_3',
       'ins_summit_max', 'notscWe_sub', 'notscWe_ex', 'notscWd_sub',
       'notscWd_ex', 'sc_sub', 'sc_ex', 'pub_avg', 'pub_sd', 'n_ex',
       'n_pkg', 'n_pra_ex_avg', 'n_pra_ex_sd','delta_days']


# shuffle the dataset
df_test = df_part.sample(frac=0.008)
# choose the test dataset in the left & shuffle
df_train = df_part[[False if i in df_test.index else True for i in range(len(df_part))]].sample(frac=0.024)

test_label = df_part[[True if i in df_test.index.values else False for i in df_part.index.values]].y.values
test_label = [int(label) for label in test_label]
train_label = df_part[[True if i in df_train.index.values else False for i in df_part.index.values]].y.values
train_label = [int(label) for label in train_label]
df_test[LABEL_COLUMN] = test_label
df_train[LABEL_COLUMN] = train_label
df_train = df_train.reset_index().drop(['index'], axis=1)
df_test = df_test.reset_index().drop(['index'], axis=1)
df_train['top_schl_id'] = [str(val) for val in df_train.top_schl_id.values]
df_test['top_schl_id'] = [str(val) for val in df_test.top_schl_id.values]
df_train['year'] = [str(val) for val in df_train.year.values]
df_test['year'] = [str(val) for val in df_test.year.values]



def input_fn(df):
    # create a dictonary mapping from each continues features column name(k) 
    # to the values of that column stored in a constant tensor
    continuous_cols = {k: tf.constant(df[k].values) 
                       for k in CONTINUES_COLUMNS}
    
    # create a dictonary mapping from each catrgorical features column name(k) 
    # to the values of that column stored in a constant tensor
    categorical_cols = {
        k: tf.SparseTensor(
            indices = [[i,0] for i in range(df[k].size)],
            values = df[k].values,
            dense_shape = [df[k].size, 1]
        )
        for k in CATEGORY_COLUMNS
    }
    
    # merge the two dictionaries into one
    feature_cols = dict(continuous_cols.items() + categorical_cols.items())
    # converts the labels column into a constant tensor
    label = tf.constant(df[LABEL_COLUMN].values)
    # return the feature columns and the label
    return feature_cols, label   


def train_input_fn():
    return input_fn(df_train)

def test_input_fn():
    return input_fn(df_test)    

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c3931fe50>, '_model_dir': '/var/folders/p2/4dg2st2j5dq310jd40nwwcdw0000gn/T/tmpt1CpLZ', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_save_summary_steps': 100, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_log_step_count_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}


In [25]:
m.fit(input_fn = train_input_fn,steps=200)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 2 into /var/folders/p2/4dg2st2j5dq310jd40nwwcdw0000gn/T/tmpt1CpLZ/model.ckpt.
INFO:tensorflow:loss = 46.180183, step = 2
INFO:tensorflow:global_step/sec: 26.5794
INFO:tensorflow:loss = 45.33406, step = 202 (6.594 sec)
INFO:tensorflow:Saving checkpoints for 202 into /var/folders/p2/4dg2st2j5dq310jd40nwwcdw0000gn/T/tmpt1CpLZ/model.ckpt.
INFO:tensorflow:Loss for final step: 45.33406.


DNNLinearCombinedClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinaryLogisticHead object at 0x1c3931fd50>, 'linear_optimizer': None, 'embedding_lr_multipliers': None, 'dnn_activation_fn': <function relu at 0x10f677c80>, 'dnn_optimizer': None, 'joint_linear_weights': False, 'dnn_feature_columns': (_RealValuedColumn(column_name='par_days', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='actual_day', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='sub_tol', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='sub_avg', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='sub_avg_a', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='ex_tol', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), 

In [26]:
results = m.evaluate(input_fn=test_input_fn,steps=1)

INFO:tensorflow:Starting evaluation at 2018-06-11-08:12:11
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/p2/4dg2st2j5dq310jd40nwwcdw0000gn/T/tmpt1CpLZ/model.ckpt-202
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2018-06-11-08:12:13
INFO:tensorflow:Saving dict for global step 202: accuracy = 0.8840116, accuracy/baseline_label_mean = 0.8848837, accuracy/threshold_0.500000_mean = 0.8840116, auc = 0.49917623, auc_precision_recall = 0.93815255, global_step = 202, labels/actual_label_mean = 0.8848837, labels/prediction_mean = 0.9944089, loss = 36.40611, precision/positive_threshold_0.500000_mean = 0.88478327, recall/positive_threshold_0.500000_mean = 0.99901444


In [27]:
results

{'accuracy': 0.8840116,
 'accuracy/baseline_label_mean': 0.8848837,
 'accuracy/threshold_0.500000_mean': 0.8840116,
 'auc': 0.49917623,
 'auc_precision_recall': 0.93815255,
 'global_step': 202,
 'labels/actual_label_mean': 0.8848837,
 'labels/prediction_mean': 0.9944089,
 'loss': 36.40611,
 'precision/positive_threshold_0.500000_mean': 0.88478327,
 'recall/positive_threshold_0.500000_mean': 0.99901444}